In [1]:
from src.data import extract_vault, holdout
import matplotlib.pyplot as plt 
import networkx as nx
import numpy as np
from src.model_base import calculate_adamic_adar, create_pairs_for_nodes, generate_recomendations
from tqdm import tqdm
from copy import deepcopy

## Baseline prediction and scoring

In [2]:
vault = extract_vault('./vaultes/brain/')

<class 'pandas.core.frame.DataFrame'>
Index: 135 entries, pure CSS content toggle study to placeholder-1671408403833.jpeg
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   rel_filepath      124 non-null    object        
 1   abs_filepath      124 non-null    object        
 2   file_exists       135 non-null    object        
 3   n_backlinks       135 non-null    object        
 4   n_wikilinks       114 non-null    float64       
 5   n_tags            114 non-null    float64       
 6   n_embedded_files  114 non-null    float64       
 7   modified_time     124 non-null    datetime64[ns]
 8   graph_category    135 non-null    object        
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 10.5+ KB
None


In [3]:
train_graph, holdout_links = holdout(vault.graph, seed=42, alpha=0.1)
sum([hl in train_graph.edges for hl in holdout_links])

1

In [5]:
for hl in holdout_links:
    print(hl[0])

Hypothesis review
learn technical art
hashtag synonyms
Obsidian improvements
BEM naming convention
1 directional annotations
Obsidian private comments
Obsidian private comments
interwikilinks plugin
wikilinks in code comments
visualize your progress
Create a site with mkdocs
plugin ideas for Obsidian


In [6]:
holdout_nodes = [hl[0] for hl in holdout_links]
pairs = create_pairs_for_nodes(nodes=holdout_nodes, graph=train_graph)
scores = calculate_adamic_adar(graph=train_graph, pair_nodes=pairs).groupby('node_from').agg(lambda x: list(x))

In [7]:
recs = generate_recomendations(scores)

In [8]:
recs

,recomendations
note_id,
1 directional annotations,"[add notes to explorer, Obsidian, Obsidian imp..."
BEM naming convention,"[CSS collapse heading, subsequent-sibling comb..."
Create a site with mkdocs,"[Component Object Model, Obsidian Web - Chrome..."
Hypothesis review,"[Obsidian, Obsidian private comments, add note..."
Obsidian improvements,"[plugin ideas for Obsidian, Obsidian, 1 direct..."
Obsidian private comments,"[add notes to explorer, add notes to explorer,..."
hashtag synonyms,"[redirect for notes, setup Obsidian Git, Obsid..."
interwikilinks plugin,"[wikilinks, obsidian web integration, Obsidian..."
learn technical art,"[build a portfolio, TA brings value to the tea..."


In [9]:
import pandas as pd
holdout_links_df = pd.DataFrame.from_records(holdout_links, columns=['note_id', 'notes_to', 'p']).groupby('note_id').agg(lambda x: list(x))[['notes_to']]

In [10]:
holdout_links_df

,notes_to
note_id,
1 directional annotations,[save quotes n lines]
BEM naming convention,[A Philosophy of Software Design]
Create a site with mkdocs,[Obsidian]
Hypothesis review,[annotate websites]
Obsidian improvements,[Obsidian]
Obsidian private comments,"[Obsidian private comments-1673286147963.jpeg,..."
hashtag synonyms,[wikilinks]
interwikilinks plugin,[Wikipedia:Main Page]
learn technical art,[goal driven learning]


In [11]:
from src.metrics import recall_n

In [13]:
recall_n(recs=recs, holdout_links=holdout_links_df, topn=10)

0.25

## Evaluate on different splittings

In [ ]:
graph = vault.graph

recall = []
for _ in tqdm(range(100))
    train_graph, holdout_links = holdout(graph, seed=None, alpha=0.1)
    sum([hl in train_graph.edges for hl in holdout_links])

    holdout_nodes = [hl[0] for hl in holdout_links]
    pairs = create_pairs_for_nodes(nodes=holdout_nodes, graph=train_graph)
    scores = calculate_adamic_adar(graph=train_graph, pair_nodes=pairs).groupby('node_from').agg(lambda x: list(x))

    recs = generate_recomendations(scores)

    holdout_links_df = pd.DataFrame.from_records(holdout_links, columns=['note_id', 'notes_to', 'p']).groupby('note_id').agg(lambda x: list(x))[['notes_to']]

    recall.append(recall_n(recs=recs, holdout_links=holdout_links_df, topn=5))